## Recipe Scraper for allrecipes.com

### Input of the model
#### 1. homepage
	- This is the recipe website. 
    - eg. 'https://www.allrecipes.com/recipes/78/breakfast-and-brunch/'
#### 2. numofpage
    - This is the maximum number of pages. 
    - eg. 'https://www.allrecipes.com/recipes/78/breakfast-and-brunch/?page=194'
#### 3. path
    - This is the path you want to save recipe data. 
    - eg. ../data/breakfast
#### 4. group_name
    - This is the category of scraped recipes. It should be determined by the programmer manually. 
    - eg. breakfast
    
By changing above 4 input, the recipe data can be scraped and saved.

In [1]:
from urllib.request import urlopen, urljoin
from urllib import error
from urllib.parse import urlparse

from urllib.robotparser import RobotFileParser

import requests
from bs4 import BeautifulSoup
import re
import os

import pandas as pd
from pandas import DataFrame
import numpy as np
import json
import time

In [2]:
def geturls(page):
    dietpage = urlopen(recipepage[page])
    dietsoup = BeautifulSoup(dietpage,'html.parser') 
    urls = []
    recipecard = dietsoup.find_all('div', attrs = {'class':'recipe-card recipeCard'})
    for i in range(len(recipecard)):
        recipe = recipecard[i]
        recipelink = recipe.find_all('a', attrs = {'class':'recipeCard__imageLink'})
        for link in recipelink:
            urls.append(link.get('href'))
    return urls

In [3]:
'''modify'''
homepage = 'https://www.allrecipes.com/recipes/78/breakfast-and-brunch/' # input
numofpage = 194 # input

recipepage = []
recipepage.append(homepage) 

for i in range(1,numofpage): 
    recipepage.append(homepage+'?page='+str(i+1)) 

print(recipepage)
    
urls = []
for i in range(len(recipepage)):
    for m in range(10):
        try:
            add = geturls(i)
            print('get page '+str(i)+' all the recipe urls')
            time.sleep(0.1)
            break
        except Exception as e:
            if m >= 9:
                print(e)
            else:
                time.sleep(0.1)
    urls = urls + add

['https://www.allrecipes.com/recipes/731/world-cuisine/european/greek/', 'https://www.allrecipes.com/recipes/731/world-cuisine/european/greek/?page=2', 'https://www.allrecipes.com/recipes/731/world-cuisine/european/greek/?page=3', 'https://www.allrecipes.com/recipes/731/world-cuisine/european/greek/?page=4', 'https://www.allrecipes.com/recipes/731/world-cuisine/european/greek/?page=5', 'https://www.allrecipes.com/recipes/731/world-cuisine/european/greek/?page=6', 'https://www.allrecipes.com/recipes/731/world-cuisine/european/greek/?page=7', 'https://www.allrecipes.com/recipes/731/world-cuisine/european/greek/?page=8', 'https://www.allrecipes.com/recipes/731/world-cuisine/european/greek/?page=9', 'https://www.allrecipes.com/recipes/731/world-cuisine/european/greek/?page=10', 'https://www.allrecipes.com/recipes/731/world-cuisine/european/greek/?page=11', 'https://www.allrecipes.com/recipes/731/world-cuisine/european/greek/?page=12', 'https://www.allrecipes.com/recipes/731/world-cuisine/e

In [4]:
class recipeinfo:
    '''Yes'''
    def __init__(self, url):
        for i in range(10):
            try:
                self.recipepage = urlopen(url)
                time.sleep(0.1)
                break            
            except Exception as e:
                if i >= 9:
                    print(e)
                else:
                    time.sleep(0.1)
        self.recipesoup = BeautifulSoup(self.recipepage,'html.parser')
        
    '''Yes'''
    def getname(self):
        self.name = self.recipesoup.find(attrs = {'class':'headline heading-content'})
        try:
            self.name = self.name.string
        except AttributeError:
            self.name = 'unknown'
        self.name = self.name.replace('/',' or ')
        self.name = self.name.replace('"', '')
        return self.name
        
    
    '''Yes'''
    def getsummary(self):
        try:
            self.summary = self.recipesoup.find(attrs = {'class':'recipe-summary margin-8-bottom'})
        except Exception as e:
            print(e)
            self.summary = 'not known'
        try:
            self.paragraph = self.summary.find(attrs = {'class':'margin-0-auto'})
            self.paragraph = self.paragraph.string
        except Exception as e:
            print(e)
            self.paragraph = 'not known'
        return self.paragraph
    
    '''Yes'''
    def getauthor(self):
        self.author = self.recipesoup.find(attrs = {'class':'author-name link'})
        try: 
            self.author = self.author.string
        except AttributeError:
            self.author = 'unknown'
        try:
            self.authorlink = self.author.get('href')
        except AttributeError:
            self.authorlink = 'unknown'
        return self.author
        return self.authorlink
    
    '''Yes'''
    def getrecipeinfo(self):
        try:
            self.recipeinfo = self.recipesoup.find_all(attrs = {'class':'recipe-meta-item-body'})
            self.infolist = []
            for info in self.recipeinfo:
                info = info.string
                info = info.replace('\n','')
                info = re.sub(' +', ' ', info)
                info = info.strip()
                self.infolist.append(info)
            self.infolist[-2] = int(self.infolist[-2])
        except Exception as e:
            print(e)
            self.infolist = [1,'none']
        return self.infolist
    
    '''Yes'''
    def getingredients(self):
        try:
            self.ingredients = self.recipesoup.find_all(attrs = {'class':'ingredients-item-name'})
            self.ingredientslist = []
            for ingredient in self.ingredients:
                self.ingredientslist.append(ingredient.string)

            for i in range(len(self.ingredientslist)):
                self.ingredientslist[i] = self.ingredientslist[i].replace('\n','')
                self.ingredientslist[i] = re.sub(' +', ' ', self.ingredientslist[i])
                self.ingredientslist[i] = self.ingredientslist[i].strip()
        except Exception as e:
            print(e)
            self.ingredientslist = ['not known']
        return self.ingredientslist

    '''Yes'''
    def getdirections(self):
        try:
            self.directions = self.recipesoup.find_all(attrs = {'class':'subcontainer instructions-section-item'})
            self.directionslist = []
            for direction in self.directions:
                direction = direction.find('div')
                direction = direction.find('p')
                direction = direction.string
                self.directionslist.append(direction)
        except Exception as e:
            print(e)
            self.directionslist = ['not known']
        return self.directionslist
    
    '''Yes'''
    def getnutrition(self):
        try:
            self.nutrition = self.recipesoup.find(attrs = {'class':'partial recipe-nutrition-section'})
            self.nutrition = self.nutrition.find(attrs = {'class':'section-body'})
            self.nutrition = self.nutrition.text
            self.nutrition = self.nutrition.replace('Full Nutrition\n','')
            self.nutrition = re.sub(' +', ' ', self.nutrition)
            self.nutrition = self.nutrition.strip()
            self.nutritionlist = self.nutrition.split(';')
        except Exception as e:
            print(e)
            self.nutritionlist = [0,0,0,0,0,0]
        self.nutritionfact = []
        for i in range(len(self.nutritionlist)):
            if i == 0:
                self.nutritionlist[i] = self.nutritionlist[i].replace(' calories','')
                try:
                    self.nutritionlist[i] = float(self.nutritionlist[i])
                except Exception:
                    self.nutritionlist[i] = 0
                self.nutritionfact.append(self.nutritionlist[i])
            if i == 1:
                self.nutritionlist[i] = self.nutritionlist[i].replace(' protein ','')
                self.nutritionlist[i] = self.nutritionlist[i].replace(' DV','')
                temp = self.nutritionlist[i].split('g')
                try:  
                    temp[1] = temp[1].strip()
                    temp[0] = float(temp[0])
                except Exception:
                    temp[0] = 0
                self.nutritionfact.append(temp[0])
            if i == 2:
                self.nutritionlist[i] = self.nutritionlist[i].replace(' carbohydrates ','')
                self.nutritionlist[i] = self.nutritionlist[i].replace(' DV','')
                temp = self.nutritionlist[i].split('g')
                try:  
                    temp[1] = temp[1].strip()
                    temp[0] = float(temp[0])
                except Exception:
                    temp[0] = 0
                self.nutritionfact.append(temp[0])
            if i == 3:
                self.nutritionlist[i] = self.nutritionlist[i].replace(' fat ','')
                self.nutritionlist[i] = self.nutritionlist[i].replace(' DV','')
                temp = self.nutritionlist[i].split('g')
                try:  
                    temp[1] = temp[1].strip()
                    temp[0] = float(temp[0])
                except Exception:
                    temp[0] = 0
                self.nutritionfact.append(temp[0])
            if i == 4:
                self.nutritionlist[i] = self.nutritionlist[i].replace(' cholesterol ','')
                self.nutritionlist[i] = self.nutritionlist[i].replace(' DV','')
                temp = self.nutritionlist[i].split('mg')
                try:  
                    temp[1] = temp[1].strip()
                    temp[0] = float(temp[0])
                except Exception:
                    temp[0] = 0
                self.nutritionfact.append(temp[0])
            if i == 5:
                self.nutritionlist[i] = self.nutritionlist[i].replace(' sodium ','')
                self.nutritionlist[i] = self.nutritionlist[i].replace(' DV','')
                temp = self.nutritionlist[i].split('mg')
                try:  
                    temp[1] = temp[1].strip()
                    temp[0] = float(temp[0])
                except Exception:
                    temp[0] = 0
                self.nutritionfact.append(temp[0])
        return self.nutritionfact
    

In [5]:
# Each website is a separate project (folder)
def create_project_dir(directory):
    if not os.path.exists(directory):
        print('Creating directory ' + directory)
        os.makedirs(directory)

In [6]:
'''modify'''
path = '../data/breakfast' # input
group_name = 'breakfast' # input
create_project_dir(path) # '../data/breakfast'
create_project_dir(path + '/each_'group_name)  # '../data/breakfast/each_breakfast'
output = []
iteration = 0

Creating directory GreekFood
Creating directory GreekFood/GreekFoodData


In [7]:
'''modify'''
def recipedata(start,urls,path,group_name):
    '''########################## Make changes ############################'''
    global iteration
    global output
    table = pd.DataFrame(columns = [
                            'model','id','recipe_name','servings','group_name','calories',
                            'protein','carbohydrates','fat','ingredients','recipe_source'
                        ])

    each = pd.DataFrame(columns = [
                            'model','id','recipe_name','servings','group_name','calories',
                            'protein','carbohydrates','fat','ingredients','recipe_source'
                        ])    
    
    for recipeid in range(start,len(urls)):
        iteration = recipeid
        print('Scraping the '+str(recipeid)+'th' + ' recipe')
    
        information = recipeinfo(urls[recipeid])
        information.getname()
        information.getsummary()
        information.getauthor()
        information.getrecipeinfo()
        information.getingredients()
        information.getdirections()
        information.getnutrition()
            
        data = {'model': 'recipes.recipe',
                'id':recipeid+1,
                'recipe_name': information.name,
                'servings': information.infolist[-2],
                'group_name': group_name,
                'calories': information.nutritionfact[0],
                'protein': information.nutritionfact[1],
                'carbohydrates': information.nutritionfact[2],
                'fat': information.nutritionfact[3], 
                'ingredients':information.ingredientslist,
                'recipe_source':urls[recipeid]
               }
        output.append(data)
        print('the len of urls is '+ str(len(urls))+'. the len of output is '+ str(len(output))+'.')
        
        print('Save the accumulated recipes until the '+str(recipeid)+'th' + ' recipe to JSON format')
        with open(path+'/'+group_name+'.json', mode='w', encoding='utf-8') as f: 
            json.dump(output, f)
        
        print('Save the '+str(recipeid)+'th' + ' recipe to CSV format')
        
        each = each.append([{'recipe':'breakfast-' + str(recipeid+1),'model':'recipes.recipe','id':recipeid+1,
                               'recipe_name':information.name,'servings':information.infolist[-2],
                               'group_name':group_name,
                               'calories':information.nutritionfact[0],
                               'protein':information.nutritionfact[1],
                               'carbohydrates':information.nutritionfact[2],
                               'fat':information.nutritionfact[3],
                               'ingredients':information.ingredientslist,
                               'recipe_source':urls[recipeid]}], ignore_index=True)
        
        each.to_csv(path + '/each_'+group_name+'/'+information.name+'.csv')
        each = pd.DataFrame(columns = [
                                'model','id','recipe_name','servings','group_name','calories',
                                'protein','carbohydrates','fat','ingredients','recipe_source'
                            ])
        print('Save the accumulated recipes until the '+str(recipeid)+'th' + ' recipe to CSV format')
        table = table.append([{'recipe':'breakfast-' + str(recipeid+1),'model':'recipes.recipe','id':recipeid+1,
                               'recipe_name':information.name,'servings':information.infolist[-2],
                               'group_name':group_name,
                               'calories':information.nutritionfact[0],
                               'protein':information.nutritionfact[1],
                               'carbohydrates':information.nutritionfact[2],
                               'fat':information.nutritionfact[3],
                               'ingredients':information.ingredientslist,
                               'recipe_source':urls[recipeid]}], ignore_index=True)
        table.to_csv(path+'/'+group_name+'.csv')
        if len(output) >= len(urls):
            break
        
    print('Save all recipes as JSON file')
    with open(path+'/'+group_name+'.json', mode='w', encoding='utf-8') as f:
        json.dump(output, f)
        # 将字典列表存入json文件中

In [9]:
'''modify'''
while True:
    print('the len of urls is '+ str(len(urls))+'. the len of output is '+ str(len(output))+'.')
    if len(output) >= len(urls):
        break
    try:
        print('The input iteration is '+str(iteration))
        print('The len of urls is '+ str(len(urls)))
        recipedata(iteration,urls,path,group_name)
    except Exception as e:
        print(e)
        print('The current iter is '+str(iteration)+'. Now input to RecipeJson again.')
        urls.pop(iteration)
        print('The problematic elements in urls has been removed. The len of urls is now '+str(len(urls))+'.')

        with open(path+'/'+group_name+'.json', mode='r', encoding='utf-8') as f:
            output = json.load(f)

The input iteration is 0
The len of urls is 495
Scraping the 0th recipe
Save the accumulated recipes until the 0th recipe to JSON format
Save the 0th recipe to CSV format
Scraping the 1th recipe
Save the accumulated recipes until the 1th recipe to JSON format
Save the 1th recipe to CSV format
Scraping the 2th recipe
Save the accumulated recipes until the 2th recipe to JSON format
Save the 2th recipe to CSV format
Scraping the 3th recipe
Save the accumulated recipes until the 3th recipe to JSON format
Save the 3th recipe to CSV format
Scraping the 4th recipe
Save the accumulated recipes until the 4th recipe to JSON format
Save the 4th recipe to CSV format
Scraping the 5th recipe
Save the accumulated recipes until the 5th recipe to JSON format
Save the 5th recipe to CSV format
Scraping the 6th recipe
Save the accumulated recipes until the 6th recipe to JSON format
Save the 6th recipe to CSV format
Scraping the 7th recipe
Save the accumulated recipes until the 7th recipe to JSON format
Sa

Save the accumulated recipes until the 65th recipe to JSON format
Save the 65th recipe to CSV format
Scraping the 66th recipe
Save the accumulated recipes until the 66th recipe to JSON format
Save the 66th recipe to CSV format
Scraping the 67th recipe
Save the accumulated recipes until the 67th recipe to JSON format
Save the 67th recipe to CSV format
Scraping the 68th recipe
Save the accumulated recipes until the 68th recipe to JSON format
Save the 68th recipe to CSV format
Scraping the 69th recipe
Save the accumulated recipes until the 69th recipe to JSON format
Save the 69th recipe to CSV format
Scraping the 70th recipe
Save the accumulated recipes until the 70th recipe to JSON format
Save the 70th recipe to CSV format
Scraping the 71th recipe
Save the accumulated recipes until the 71th recipe to JSON format
Save the 71th recipe to CSV format
Scraping the 72th recipe
Save the accumulated recipes until the 72th recipe to JSON format
Save the 72th recipe to CSV format
Scraping the 73th

Save the accumulated recipes until the 130th recipe to JSON format
Save the 130th recipe to CSV format
Scraping the 131th recipe
Save the accumulated recipes until the 131th recipe to JSON format
Save the 131th recipe to CSV format
Scraping the 132th recipe
Save the accumulated recipes until the 132th recipe to JSON format
Save the 132th recipe to CSV format
Scraping the 133th recipe
Save the accumulated recipes until the 133th recipe to JSON format
Save the 133th recipe to CSV format
Scraping the 134th recipe
Save the accumulated recipes until the 134th recipe to JSON format
Save the 134th recipe to CSV format
Scraping the 135th recipe
Save the accumulated recipes until the 135th recipe to JSON format
Save the 135th recipe to CSV format
Scraping the 136th recipe
Save the accumulated recipes until the 136th recipe to JSON format
Save the 136th recipe to CSV format
Scraping the 137th recipe
Save the accumulated recipes until the 137th recipe to JSON format
Save the 137th recipe to CSV f

Save the accumulated recipes until the 194th recipe to JSON format
Save the 194th recipe to CSV format
Scraping the 195th recipe
Save the accumulated recipes until the 195th recipe to JSON format
Save the 195th recipe to CSV format
Scraping the 196th recipe
Save the accumulated recipes until the 196th recipe to JSON format
Save the 196th recipe to CSV format
Scraping the 197th recipe
Save the accumulated recipes until the 197th recipe to JSON format
Save the 197th recipe to CSV format
Scraping the 198th recipe
Save the accumulated recipes until the 198th recipe to JSON format
Save the 198th recipe to CSV format
Scraping the 199th recipe
Save the accumulated recipes until the 199th recipe to JSON format
Save the 199th recipe to CSV format
Scraping the 200th recipe
Save the accumulated recipes until the 200th recipe to JSON format
Save the 200th recipe to CSV format
Scraping the 201th recipe
Save the accumulated recipes until the 201th recipe to JSON format
Save the 201th recipe to CSV f

Save the accumulated recipes until the 258th recipe to JSON format
Save the 258th recipe to CSV format
Scraping the 259th recipe
Save the accumulated recipes until the 259th recipe to JSON format
Save the 259th recipe to CSV format
Scraping the 260th recipe
Save the accumulated recipes until the 260th recipe to JSON format
Save the 260th recipe to CSV format
Scraping the 261th recipe
Save the accumulated recipes until the 261th recipe to JSON format
Save the 261th recipe to CSV format
Scraping the 262th recipe
Save the accumulated recipes until the 262th recipe to JSON format
Save the 262th recipe to CSV format
Scraping the 263th recipe
Save the accumulated recipes until the 263th recipe to JSON format
Save the 263th recipe to CSV format
Scraping the 264th recipe
Save the accumulated recipes until the 264th recipe to JSON format
Save the 264th recipe to CSV format
Scraping the 265th recipe
Save the accumulated recipes until the 265th recipe to JSON format
Save the 265th recipe to CSV f

Save the accumulated recipes until the 322th recipe to JSON format
Save the 322th recipe to CSV format
Scraping the 323th recipe
Save the accumulated recipes until the 323th recipe to JSON format
Save the 323th recipe to CSV format
Scraping the 324th recipe
Save the accumulated recipes until the 324th recipe to JSON format
Save the 324th recipe to CSV format
Scraping the 325th recipe
Save the accumulated recipes until the 325th recipe to JSON format
Save the 325th recipe to CSV format
Scraping the 326th recipe
Save the accumulated recipes until the 326th recipe to JSON format
Save the 326th recipe to CSV format
Scraping the 327th recipe
Save the accumulated recipes until the 327th recipe to JSON format
Save the 327th recipe to CSV format
Scraping the 328th recipe
Save the accumulated recipes until the 328th recipe to JSON format
Save the 328th recipe to CSV format
Scraping the 329th recipe
Save the accumulated recipes until the 329th recipe to JSON format
Save the 329th recipe to CSV f

Save the accumulated recipes until the 386th recipe to JSON format
Save the 386th recipe to CSV format
Scraping the 387th recipe
Save the accumulated recipes until the 387th recipe to JSON format
Save the 387th recipe to CSV format
Scraping the 388th recipe
Save the accumulated recipes until the 388th recipe to JSON format
Save the 388th recipe to CSV format
Scraping the 389th recipe
Save the accumulated recipes until the 389th recipe to JSON format
Save the 389th recipe to CSV format
Scraping the 390th recipe
Save the accumulated recipes until the 390th recipe to JSON format
Save the 390th recipe to CSV format
Scraping the 391th recipe
Save the accumulated recipes until the 391th recipe to JSON format
Save the 391th recipe to CSV format
Scraping the 392th recipe
Save the accumulated recipes until the 392th recipe to JSON format
Save the 392th recipe to CSV format
Scraping the 393th recipe
Save the accumulated recipes until the 393th recipe to JSON format
Save the 393th recipe to CSV f

Save the accumulated recipes until the 450th recipe to JSON format
Save the 450th recipe to CSV format
Scraping the 451th recipe
Save the accumulated recipes until the 451th recipe to JSON format
Save the 451th recipe to CSV format
Scraping the 452th recipe
Save the accumulated recipes until the 452th recipe to JSON format
Save the 452th recipe to CSV format
Scraping the 453th recipe
Save the accumulated recipes until the 453th recipe to JSON format
Save the 453th recipe to CSV format
Scraping the 454th recipe
Save the accumulated recipes until the 454th recipe to JSON format
Save the 454th recipe to CSV format
Scraping the 455th recipe
Save the accumulated recipes until the 455th recipe to JSON format
Save the 455th recipe to CSV format
Scraping the 456th recipe
Save the accumulated recipes until the 456th recipe to JSON format
Save the 456th recipe to CSV format
Scraping the 457th recipe
Save the accumulated recipes until the 457th recipe to JSON format
Save the 457th recipe to CSV f

In [ ]:
'''length check'''
with open(path+'/'+group_name+'.json', mode = 'r', encoding = 'utf-8') as f:
    output = json.load(f)
print('now the len of output is '+ str(len(output)))    
recipe_name = []
for i in range(len(output)):
    recipe_name.append(output[i]['recipe_name'])
setrecipe_name = set(recipe_name)
print('the number of non-repeatable recipe name is' + str(len(setrecipe_name)))
if len(setrecipe_name) == len(recipe_name):
    print('no repeatative elements')
else:
    print('repeatation exists')

recipe_name = []
new = []
for i in range(len(output)):
    if output[i]['recipe_name'] in recipe_name:
        pass
    else:
        recipe_name.append(output[i]['recipe_name'])
        new.append(output[i])
for i in range(len(new)):
    new[i]['id'] = i + 1

with open(path+'/'+group_name+'.json', mode='w', encoding='utf-8') as f:
    json.dump(new,f)
        
with open(path+'/'+group_name+'.json', mode = 'r', encoding = 'utf-8') as f: 
    output = json.load(f)
print('the len of new output is '+ str(len(output))) 

In [ ]:
output_old = []
allrecipes_list =  [path+'/'+group_name+'.json'] # other json files with same format can be inserted here.
print(allrecipes_list)
for i in range(len(allrecipes_list)):
    with open(allrecipes_list[i], mode = 'r', encoding = 'utf-8') as f:
        output_old.append(json.load(f))

In [ ]:
'''this function is to merge separate json files into one json file and another csv file'''
def csvjson(allrecipes):

    output  = []
    table = pd.DataFrame(index = [
                            'model','id','recipe_name','servings','group_name','calories',
                            'protein','carbohydrates','fat','ingredients','recipe_source'
                        ])
            
    '''for allrecipes.com'''
    for i in range(len(allrecipes)):
        for recipeid in range(len(allrecipes[i])):
            data = {'model': 'recipes.recipe',
                    'id':recipeid+1,
                    'recipe_name': allrecipes[i][recipeid]['recipe_name'],
                    'servings': int(allrecipes[i][recipeid]['servings']),
                    'group_name': allrecipes[i][recipeid]['group_name'],
                    'calories': float(allrecipes[i][recipeid]['calories']),
                    'protein': float(allrecipes[i][recipeid]['protein']),
                    'carbohydrates': float(allrecipes[i][recipeid]['carbohydrates']),
                    'fat': float(allrecipes[i][recipeid]['fat']),
                    'ingredients': allrecipes[i][recipeid]['ingredients'],
                    'recipe_source': allrecipes[i][recipeid]['recipe_source'],
                   }
            output.append(data)     

            table[allrecipes[i][recipeid]['group_name']+'-id'+str(recipeid+1)] = [
                'recipes.recipe',recipeid+1,allrecipes[i][recipeid]['recipe_name'],int(allrecipes[i][recipeid]['servings']),
                allrecipes[i][recipeid]['group_name'],
                float(allrecipes[i][recipeid]['calories']),
                float(allrecipes[i][recipeid]['protein']),
                float(allrecipes[i][recipeid]['carbohydrates']),
                float(allrecipes[i][recipeid]['fat']),allrecipes[i][recipeid]['ingredients'],
                allrecipes[i][recipeid]['recipe_source']         
            ]
    
    table = table.stack()
    table = table.unstack(0)
    table.to_csv(path+'/'+group_name+'.csv')
        
    print('Save as JSON file')
    with open(path+'/'+group_name+'.json', mode='w', encoding='utf-8') as f:
        json.dump(output, f)

In [ ]:
csvjson(output_old) 
with open(path+'/'+group_name+'.json', mode='r', encoding='utf-8') as f:
    output = json.load(f)
print(len(output))